# **Nivel SparkSQL**

**Instalação do PySpark**

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 66.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=46fcd8cf4ca234ef8fafe35b450ed115dc07d62a65a81a65e3c475f84c605fc3
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


**Importação de biliotecas**

In [12]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

**Conecção com o Apache-Spark**

In [13]:
spark = (SparkSession.builder\
        .master("local")\
        .appName("market_camp")\
        .config("spark.ui.port", "4050")\
        .getOrCreate())

**Carrega dados do arquivo - df_pd_marketing_campaign.csv**

In [14]:
df = (spark
      .read
      .format("csv")
      .option("header", "true")
      .option("inferschema", "true")
      .option("delimiter", ",")
      .load("/content/df_pd_marketing_campaign.csv")
      .createOrReplaceTempView("marketing"))

**1 - Imprime a tabela na tela 10 linhas.**

In [17]:
spark.sql('''SELECT * FROM marketing''').show(10)

+---+----+--------------+---------+-------------+-----------+-----------+---------------+--------------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+----------------+-------+--------+
|_c0|  id|ano_nascimento| educacao| estado_civil|renda_anual|qtd_crianca|qtd_adolescente|primeira_data_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|contato_de_custo|receita|resposta|
+---+----+--------------+---------+-------------+-----------+-----------+---------------+--------------------+----------------------+----------+--

**2 - Imprime os indexes da tabela.**

In [16]:
spark.sql('''DESCRIBE marketing''').show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|                 _c0|      int|   null|
|                  id|      int|   null|
|      ano_nascimento|      int|   null|
|            educacao|   string|   null|
|        estado_civil|   string|   null|
|         renda_anual|   double|   null|
|         qtd_crianca|      int|   null|
|     qtd_adolescente|      int|   null|
|primeira_data_compra|   string|   null|
|dias_desde_ult_co...|      int|   null|
|          qtd_vinhos|      int|   null|
|          qtd_frutas|      int|   null|
|           qtd_carne|      int|   null|
|           qtd_peixe|      int|   null|
|          guloseimas|      int|   null|
|   qtd_produtos_gold|      int|   null|
|  qtd_ofertas_compra|      int|   null|
|  qtd_compras_online|      int|   null|
|qtd_compras_catalogo|      int|   null|
| qtd_compras_na_loja|      int|   null|
+--------------------+---------+-------+
only showing top

**3 - Seleção da renda anual e perfil de compras com base nos dados socio-economicos**

In [33]:
spark.sql('''
            SELECT id, renda_anual,  qtd_compras_online, educacao, estado_civil, dias_desde_ult_compras, qtd_visitas_online_mes
            FROM marketing
            ORDER BY renda_anual DESC;
          ''').show(10)

+-----+-----------+------------------+---------+-------------+----------------------+----------------------+
|   id|renda_anual|qtd_compras_online| educacao| estado_civil|dias_desde_ult_compras|qtd_visitas_online_mes|
+-----+-----------+------------------+---------+-------------+----------------------+----------------------+
| 9432|   666666.0|                 3| Graduado|Uniao_estavel|                    23|                     6|
| 1503|   162397.0|                 0|Doutorado|Uniao_estavel|                    31|                     1|
| 1501|   160803.0|                 0|Doutorado|       Casado|                    21|                     0|
| 5336|   157733.0|                 1| Mestrado|Uniao_estavel|                    37|                     1|
| 8475|   157243.0|                 0|Doutorado|       Casado|                    98|                     0|
| 4931|   157146.0|                 0| Graduado|Uniao_estavel|                    13|                     1|
|11181|   156924.0|

**4 - Estudo com criterio de quantdade de compras, renda anual e dias desde da ultima compra.**

In [34]:
spark.sql('''
            SELECT id, qtd_compras_online, renda_anual, dias_desde_ult_compras
            FROM marketing
            ORDER BY qtd_compras_online DESC;
          ''').show(10)

+-----+------------------+-----------------+----------------------+
|   id|qtd_compras_online|      renda_anual|dias_desde_ult_compras|
+-----+------------------+-----------------+----------------------+
| 5255|                27|52247.25135379061|                    19|
| 4619|                27|         113734.0|                     9|
|10311|                25|           4428.0|                     0|
| 6237|                23|           7144.0|                    92|
| 8931|                11|          83033.0|                    82|
| 7030|                11|          66465.0|                     1|
| 9365|                11|          60000.0|                    57|
| 6439|                11|          61074.0|                    37|
| 4299|                11|          70971.0|                    28|
| 7381|                11|          75693.0|                    10|
+-----+------------------+-----------------+----------------------+
only showing top 10 rows



**5 -  Estudo com critério do maior numero de dias sem comprar.**

In [41]:
spark.sql('''
            SELECT id, dias_desde_ult_compras, qtd_compras_online, renda_anual 
            FROM marketing
            ORDER BY dias_desde_ult_compras DESC;
          ''').show()

+-----+----------------------+------------------+-----------+
|   id|dias_desde_ult_compras|qtd_compras_online|renda_anual|
+-----+----------------------+------------------+-----------+
|  868|                    99|                 2|    44794.0|
| 7829|                    99|                 2|    36640.0|
| 7232|                    99|                 1|    42429.0|
|10142|                    99|                 5|    66476.0|
| 9977|                    99|                 9|    78901.0|
| 8595|                    99|                 1|    42429.0|
| 7947|                    99|                 1|    42231.0|
| 5263|                    99|                 1|    31056.0|
| 3363|                    99|                 1|    20130.0|
| 4070|                    99|                 8|    94871.0|
| 2831|                    99|                 7|    78416.0|
| 2415|                    99|                 5|    62568.0|
| 1743|                    99|                 2|    69719.0|
| 7212| 